# Benchmark with conditional formating
All we need in the protocal is the Haar-random unitaries. A random unitary can be represeted by rotation matrices like this:

In this project we benchmark with those conditional probabilities

In [13]:
from pyquil import get_qc, Program
from pyquil.gates import *
from pyquil.api import local_forest_runtime
from pyquil.quilbase import Declare
from pyquil.simulation.tools import lifted_gate, program_unitary, permutation_arbitrary
from pyquil.quil import *

In [14]:
import numpy as np
import math
from math import pi
import random
import copy
from tqdm import tqdm_notebook as tqdm

In [15]:
from functions import *

In [60]:
if __name__ == "__main__":
    target_qubits = [0,1]

#     First step choose m and the K_m sequences of 
    m = 2
    k_m = 10 #n. of diff sequences
    n_m = 50  #n. of samples from a certain sequence


In [61]:
def universal_two_qubits_packs_generator(qmachine, target_qubits:list, num_layer:int):
    list_gates = []
    for index in range(num_layer):
        draft_circuit = give_random_two_qubit_circuit(target_qubits)
        list_gates.extend( qmachine.compiler.quil_to_native_quil(draft_circuit) )
    list_gates = [ ins for ins in list_gates if isinstance(ins, Gate)]
    return list_gates

In [66]:
def machine_response_rb_universal_two_qubits_conditional(qmachine, target_qubits:list, m:int, k_m, n_m):
    """
    It samples and record the accept or reject of the machine with native gates chosen with conditions for rigetti.
    ::return response_matrix including accepts and rejects in columns
    """
    num_qubits = len(target_qubits)
    response_matrix = np.zeros((k_m,n_m))
    
    for i_sequ in tqdm(range(k_m), desc = 'Sequences'):
        gate_list = universal_two_qubits_packs_generator(qmachine, target_qubits, m)
        prog = Program() #All qubits begin with |0> state
        
        for gate in gate_list:
            prog += gate
        
        #Come back to our initial state
        u_inverse_definition = DefGate('U_inverse', np.linalg.inv( program_unitary(prog, n_qubits=2) ) )
        U_inverse = u_inverse_definition.get_constructor()
        prog += qmachine.compiler.quil_to_native_quil(Program(u_inverse_definition, U_inverse(0,1)))
#         prog += qmachine.compiler.quil_to_native_quil(Program(u_inverse_definition, U_inverse(*target_qubits)))
        #Do not let the quilc to alter the gates by optimization
        prog = Program('PRAGMA PRESERVE_BLOCK') + prog
        prog += Program('PRAGMA END_PRESERVE_BLOCK')
        
        #Measurments
        ro = prog.declare('ro', 'BIT', num_qubits)
        for q in range(num_qubits):
            prog += MEASURE(target_qubits[q], ro[q])
        prog = prog.wrap_in_numshots_loop(n_m)

        #Run the program
        executable = qmachine.compile(prog)
        result = qmachine.run(executable)
        measured_outcome = result.readout_data.get('ro')

        response_matrix[i_sequ,:] = 1 - np.bool_(np.sum(measured_outcome, axis = 1)) # 1 if it is equal to n_zero state
    return prog, response_matrix

In [71]:
if __name__ == "__main__":
    qc = get_qc( str(2) + 'q-qvm')  # You can make any 'nq-qvm'
#     qc = get_qc("9q-square-noisy-qvm")
    prog, response = machine_response_rb_universal_two_qubits_conditional(qc, [0,1], m, k_m, n_m)

C:\Users\mohsen\AppData\Local\Temp\ipykernel_15284\286071151.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i_sequ in tqdm(range(k_m), desc = 'Sequences'):


Sequences:   0%|          | 0/10 [00:00<?, ?it/s]

In [32]:
# if __name__ == "__main__":
#     !jupyter nbconvert RB_with_Rigetti_native_gates_conditional_probability_two_qubits.ipynb --to python

[NbConvertApp] Converting notebook RB_with_Rigetti_native_gates_conditional_probability_two_qubits.ipynb to python
[NbConvertApp] Writing 3699 bytes to RB_with_Rigetti_native_gates_conditional_probability_two_qubits.py


In [72]:
instructions = [ins for ins in prog.instructions if isinstance(ins, Gate)]
u = program_unitary( Program(instructions), n_qubits=2)
u_inv = np.linalg.inv(u)
u

array([[-0.73840035+6.38378239e-16j, -0.29775456-3.34039817e-01j,
        -0.06371966-1.50461743e-01j, -0.22695339-4.19902099e-01j],
       [-0.06371966+1.50461743e-01j,  0.26465262-8.98921993e-02j,
        -0.89793973+5.55111512e-17j,  0.29679166-2.82560938e-02j],
       [-0.29775456+3.34039817e-01j, -0.2345541 -1.17961196e-16j,
         0.26465262+8.98921993e-02j,  0.79449811+1.88135949e-01j],
       [-0.22695339+4.19902099e-01j,  0.79449811-1.88135949e-01j,
         0.29679166+2.82560938e-02j, -0.12910582-7.70217223e-16j]])

In [132]:
prog = Program(RZ(-np.pi, 0), RX(-np.pi, 1), RX(np.pi, 1), RZ(np.pi, 0))
# u_inverse_definition = DefGate('U_inverse', program_unitary(prog, n_qubits=2).conj().T )
# U_inverse = u_inverse_definition.get_constructor()
# prog_inv = qc.compiler.quil_to_native_quil(Program(u_inverse_definition, U_inverse(0,1)))

In [133]:
print(qc.compiler.quil_to_native_quil(prog))

HALT



In [131]:
print(qc.compile(prog))

RZ(-pi) 0
RX(-pi) 1
HALT



In [126]:
temp = []
for ins in prog.instructions:
    temp.extend([ins.dagger()])
temp

[<Gate DAGGER RZ(pi) 0>, <Gate DAGGER RX(pi) 1>]

In [128]:
print(qc.compile(Program(temp)))

RZ(-pi) 0
RZ(0.9134992315263939) 1
RX(pi/2) 1
RZ(pi) 1
RX(-pi/2) 1
RZ(4.055091885116186) 1
HALT

